# FEM Simulations of Diffusion in a Cylindrical Cavity

Simulates scalar diffusion in a cylindrical cavity. Code copied from [Ferrite.jl Heat Transfer Tutorial](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/heat_equation/). 

##  Import Packages

In [3]:
using BlockArrays
using LinearAlgebra
using UnPack
using LinearSolve 
using SparseArrays
using Ferrite
using FerriteGmsh 
using OrdinaryDiffEq
using DifferentialEquations
using Plots 
using WriteVTK

## Section 1: Introduction 

Simulates scalar diffusion in a cylindrical cavity. Code copied from [Ferrite.jl Heat Transfer Tutorial](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/heat_equation/). Requires more details.

## Section 2: Read 2D Mesh From External Input Mesh File  

In [9]:
grid = togrid("cylindrical_cavity.msh")

Info    : Reading 'cylindrical_cavity.msh'...
Info    : 9 entities
Info    : 1077 nodes
Info    : 5376 elements
Info    : Done reading 'cylindrical_cavity.msh'


Grid{3, Tetrahedron, Float64} with 3604 Tetrahedron cells and 1077 nodes

## Section 3: Functions for Assembly of Stiffness Matrix and Load Vector  

In [5]:
# define spatially varying diffusion coefficient 
function my_diff_coeff(x)
    return 1.  
end 

function assemble_element!(Ke::Matrix, fe::Vector, cellvalues::CellValues, mycoords)
    n_basefuncs = getnbasefunctions(cellvalues)
    # Reset to 0
    fill!(Ke, 0)
    fill!(fe, 0)
    # Loop over quadrature points
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the quadrature weight
        dΩ = getdetJdV(cellvalues, q_point)
        # ADDED: Get coord of quadrature point
        coords_qp = spatial_coordinate(cellvalues, q_point, mycoords)
        # ADDED: Evaluate spatially dependent diffusion coefficient in quad point 
        val_diff_coeff = my_diff_coeff(coords_qp)
        # ADDED: Evaluate spatially dependent source term in quad point 
        ## val_source = my_source(coords_qp)        
        # Loop over test shape functions
        for i in 1:n_basefuncs
            δu  = shape_value(cellvalues, q_point, i)
            ∇δu = shape_gradient(cellvalues, q_point, i)
            # Add contribution to fe
            fe[i] += 0. * δu * dΩ
            # Loop over trial shape functions
            for j in 1:n_basefuncs
                ∇u = shape_gradient(cellvalues, q_point, j)
                # MODIFIED: Add contribution to Ke
                Ke[i, j] += val_diff_coeff * (∇δu ⋅ ∇u) * dΩ
            end
        end
    end
    return Ke, fe
end

function assemble_global(cellvalues::CellValues, K::SparseMatrixCSC, dh::DofHandler)
    # Allocate the element stiffness matrix and element force vector
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)
    # Allocate global force vector f
    f = zeros(ndofs(dh))
    # Create an assembler
    assembler = start_assemble(K, f)
    # Loop over all cels
    for cell in CellIterator(dh)
        # Added: Get coordinates from current cell 
        coords = getcoordinates(cell)
        # Reinitialize cellvalues for this cell
        Ferrite.reinit!(cellvalues, cell)
        # Modified - Compute element contribution
        assemble_element!(Ke, fe, cellvalues, coords)
        # Assemble Ke and fe into K and f
        assemble!(assembler, celldofs(cell), Ke, fe)
    end
    return K, f
end

assemble_global (generic function with 1 method)

## Section 4: Set-up, Assembly and Solve

In [6]:
dim = 3 
degree = 1 

ip_u = Lagrange{RefTetrahedron, 1}()
qr = QuadratureRule{RefTetrahedron}(2*degree+1)
cellvalues_u = CellValues(qr, ip_u);

dh = DofHandler(grid)
add!(dh, :u, ip_u)
close!(dh);

K = allocate_matrix(dh)

ch = ConstraintHandler(dh)

∂Ω = getfacetset(grid, "lid")

dbc = Dirichlet(:u, ∂Ω, (x, t) -> x[1])
add!(ch, dbc)
close!(ch)

K, f = assemble_global(cellvalues_u, K, dh);

apply!(K, f, ch)
u = K \ f;

## Section 5: Post-Processing

In [8]:
VTKGridFile("results_cavity", dh) do vtk
    write_solution(vtk, dh, u)
end

VTKGridFile for the closed file "results_cavity.vtu".